In [10]:
import re
import itertools as it
import random as rm

In [89]:
m = re.match("c\((\d+),([a-z]{2})\)","c(3,ht)")

In [91]:
m.group(2)

'ht'

In [74]:
solution = "[c(6,sd):L1-F1,c(4,ht):L2-L1,c(4,sd):L4-L2,c(3,sd):L4-L1,c(2,ht):L4-L1,c(6,sd):F1-L4,c(2,ht):L1-F1,c(4,sd):L2-F2,c(2,ht):F1-L1,c(5,ht):L2-L4,c(1,ht):L2-G1,c(2,ht):L1-G1,c(4,sd):F2-L2,c(4,sd):L2-L4,c(3,sd):L1-L2,c(4,ht):L1-F1,c(5,sd):L1-F2,c(3,ht):L1-G1,c(4,ht):F1-G1,c(5,sd):F2-L1,c(3,sd):L2-F1,c(5,sd):L1-L2,c(3,sd):F1-L1,c(5,sd):L2-F1,c(2,sd):L3-L2,c(5,sd):F1-L3,c(3,sd):L1-F1,c(2,sd):L2-L1,c(3,sd):F1-L2,c(2,sd):L1-F1,c(3,sd):L2-L1,c(5,sd):L3-L2,c(3,sd):L1-F2,c(2,sd):F1-L1,c(5,sd):L2-F1,c(3,sd):F2-L2,c(2,sd):L1-F2,c(5,sd):F1-L1,c(3,sd):L2-F1,c(2,sd):F2-L2,c(5,sd):L1-F2,c(6,ht):L3-L1,c(1,sd):L3-G2,c(2,sd):L2-G2,c(3,sd):F1-G2,c(4,sd):L4-G2,c(5,sd):F2-G2,c(5,ht):L4-G1,c(6,ht):L1-G1,c(6,sd):L4-G2]"

In [83]:
for step in re.finditer('(c\((\d+),([a-z]{2})\)):([L,G,F]\d)-([L,G,F]\d)', solution):
    print(step.group(2))

6
4
4
3
2
6
2
4
2
5
1
2
4
4
3
4
5
3
4
5
3
5
3
5
2
5
3
2
3
2
3
5
3
2
5
3
2
5
3
2
5
6
1
2
3
4
5
5
6
6


# Generate game setups

In [116]:
clrs = ["ht","sd"]
rnks = range(1,7)

In [117]:
cards = list(map(lambda rc: f"c({rc[0]},{rc[1]})", it.product(rnks,clrs)))

In [118]:
cards = rm.sample(cards, k=len(cards))

In [119]:
l1 = str(cards[0:3])
l2 = str(cards[3:6])
l3 = str(cards[6:9])
l4 = str(cards[9:12])

In [120]:
cmd = (f"fc({l1},{l2},{l3},{l4}, S).")

In [121]:
cmd

"fc(['c(6,ht)', 'c(2,sd)', 'c(3,sd)'],['c(4,ht)', 'c(6,sd)', 'c(5,ht)'],['c(1,sd)', 'c(5,sd)', 'c(1,ht)'],['c(4,sd)', 'c(3,ht)', 'c(2,ht)'], S)."

In [122]:
print(cmd.replace("'",""))

fc([c(6,ht), c(2,sd), c(3,sd)],[c(4,ht), c(6,sd), c(5,ht)],[c(1,sd), c(5,sd), c(1,ht)],[c(4,sd), c(3,ht), c(2,ht)], S).
